In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lrex').getOrCreate()

In [2]:
from pyspark.ml.regression import LinearRegression

In [3]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [4]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [5]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [6]:
lrModel = lr.fit(training)

In [7]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
lrModel.intercept

0.14228558260358093

In [9]:
training_summary = lrModel.summary

In [10]:
training_summary.r2

0.027839179518600154

In [11]:
training_summary.rootMeanSquaredError

10.16309157133015

In [12]:
all_data = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [13]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [14]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                345|
|   mean| 0.7422538334692766|
| stddev| 10.227192897636847|
|    min|-28.046018037776633|
|    max| 27.111027963108548|
+-------+-------------------+



In [15]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                156|
|   mean|-0.8165145681231426|
| stddev| 10.468849327221388|
|    min|-28.571478869743427|
|    max|  27.78383192005107|
+-------+-------------------+



In [16]:
correct_model = lr.fit(train_data)

In [17]:
test_result = correct_model.evaluate(test_data)

In [18]:
test_result.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|-28.321480107881808|
|-28.143272307647226|
|-24.627484143418226|
|-26.057903357065083|
|-22.571408788902787|
| -20.89831942482332|
|  -20.7951797750058|
|-20.221852541513794|
| -21.19316942644216|
|-19.556001821704562|
| -19.11330711853794|
| -16.61594764421379|
|-17.758284337241182|
|-17.817936533123387|
|-20.401680381512655|
|-17.808881920772475|
| -16.30070144109888|
|-14.505473774512948|
| -15.13359713863455|
|  -14.9738084044689|
+-------------------+
only showing top 20 rows



In [19]:
unlabeled_data = test_data.select('features')

In [20]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [21]:
predictions = correct_model.transform(unlabeled_data)

In [22]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|-0.24999876186161996|
|(10,[0,1,2,3,4,5,...|  1.3377888791641532|
|(10,[0,1,2,3,4,5,...|  1.1400440224817128|
|(10,[0,1,2,3,4,5,...|   3.108077420869008|
|(10,[0,1,2,3,4,5,...|-0.26605162801655413|
|(10,[0,1,2,3,4,5,...|  0.8408368090341094|
|(10,[0,1,2,3,4,5,...|  0.9106190007323743|
|(10,[0,1,2,3,4,5,...|  0.3488615034453863|
|(10,[0,1,2,3,4,5,...|  1.5258508110704412|
|(10,[0,1,2,3,4,5,...| 0.15366579149000875|
|(10,[0,1,2,3,4,5,...|-0.05498550442582173|
|(10,[0,1,2,3,4,5,...| -0.8782527126695521|
|(10,[0,1,2,3,4,5,...|  0.4315636045652348|
|(10,[0,1,2,3,4,5,...|  1.0988396995182956|
|(10,[0,1,2,3,4,5,...|   4.140250106059925|
|(10,[0,1,2,3,4,5,...|  1.6575325694953615|
|(10,[0,1,2,3,4,5,...| 0.21504240007738856|
|(10,[0,1,2,3,4,5,...| -1.4460387912816255|
|(10,[0,1,2,3,4,5,...| 0.37083888570342194|
|(10,[0,1,2,3,4,5,...|   1.10672